In [ ]:
!pip install joblib
import joblib

from transformers import TFBertForSequenceClassification, BertTokenizer

# Load the BERT model and tokenizer
model_directory = '/content/drive/MyDrive/Hate Speech Hinglish Laguage/hate_speech_model'
loaded_model = TFBertForSequenceClassification.from_pretrained(model_directory)
loaded_tokenizer = BertTokenizer.from_pretrained(model_directory)

# Load the TensorFlow model
tf_model_filename = '/content/drive/MyDrive/Hate Speech Hinglish Laguage/tf_model.h5'
loaded_model.load_weights(tf_model_filename)

# Load the label encoder
label_encoder_filename = '/content/drive/MyDrive/Hate Speech Hinglish Laguage/label_encoder.pkl'
loaded_label_encoder = joblib.load(label_encoder_filename)


In [ ]:
import joblib
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
import numpy as np

def load_model_and_predict(text):
    # Load the BERT model and tokenizer
    model_directory = '/content/drive/MyDrive/Hate Speech Hinglish Laguage/hate_speech_model'
    loaded_model = TFBertForSequenceClassification.from_pretrained(model_directory)
    loaded_tokenizer = BertTokenizer.from_pretrained(model_directory)

    # Load the TensorFlow model
    tf_model_filename = '/content/drive/MyDrive/Hate Speech Hinglish Laguage/tf_model.h5'
    loaded_model.load_weights(tf_model_filename)

    # Load the label encoder
    label_encoder_filename = '/content/drive/MyDrive/Hate Speech Hinglish Laguage/label_encoder.pkl'
    loaded_label_encoder = joblib.load(label_encoder_filename)

    # Tokenize and preprocess the input text
    encoding = loaded_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Make prediction
    with tf.device('/cpu:0'):  # Ensure predictions are made on CPU
        outputs = loaded_model.predict([input_ids, attention_mask])
        logits = outputs.logits

    # Convert logits to probabilities and get the predicted label
    probabilities = tf.nn.softmax(logits, axis=1).numpy()[0]
    predicted_label_id = np.argmax(probabilities)
    predicted_label = loaded_label_encoder.classes_[predicted_label_id]

    return predicted_label

# Example usage:
user_input = input("Enter a text to classify: ")
predicted_label = load_model_and_predict(user_input)
print(f'Predicted Label: {predicted_label}')
